#### Question 2: Gathering Movie Data via TMDB API

* Set up the API
    * Create a free TMDB account
    * Generate an API key and review their documentation, especially:
        * /discover/movie: https://developer.themoviedb.org/reference/discover-movie
        * /movie/{movie_id}: https://developer.themoviedb.org/reference/movie-details
        * /search/movie: https://developer.themoviedb.org/reference/search-movie
* Collect top movies (2015-2024)
    * For each year from 2015 to 2024:
        * Query TMDB for the top 100 movies (by vote count).
        * For each movie, gather:
            * Title
            * Release Year
            * Genre(s)
            * Vote Average
            * Vote Count
            * Budget
            * Revenue
            * TMDB ID
* Store all results in a single DataFrame and export to movies_2015_2024.csv.
* Hint: TMDB rate limits are generous for free accounts, but you should pause between requests (eg. time.sleep(0.25)).
* Some Oscar films may not appear in the top 100 by vote count. For any missing, use the /search/movie endpoint to add it.

In [28]:
import requests
import json
import time
import pandas as pd
import datetime
from pathlib import Path 

In [17]:
# Load API key from keys file
with open('keys.json') as fi:
    credentials = json.load(fi)

api_key = credentials['api_key']

In [18]:
endpoint = 'https://api.themoviedb.org/3/discover/movie'

movie_titles = []
release_dates = []
vote_averages = []
vote_counts = []
tmdb_ids = []

# Iterate through all years between 2015 and 2024
for year in range(2015,2025):
    
    # Each page contains 20 results, so we need to iterate through 5 pages to get 100 results
    for page in range(1,6):

        # Define params
        params = {
            'api_key': api_key,
            'primary_release_year': year,
            'sort_by': 'vote_count.desc',
            'page': page
        }
    
        # Get response
        response = requests.get(endpoint, params = params)
        res = response.json()['results']
        for movie in res:
            movie_titles.append(movie['title'])
            release_dates.append(movie['release_date'])
            vote_averages.append(movie['vote_average'])
            vote_counts.append(movie['vote_count'])
            tmdb_ids.append(movie['id'])

        # Sleep before next API call 
        time.sleep(0.25)

years = [datetime.datetime.strptime(date_str, "%Y-%m-%d").year for date_str in release_dates]

In [21]:
budgets = []
revenues = []
genres = [] 
    
for movie_id in tmdb_ids:
    # Use the movie ids to search for budget, revenue, and genre information
    endpoint = f'https://api.themoviedb.org/3/movie/{movie_id}'
    # Define params
    params = {
        'api_key': api_key,
    }
    # Get response
    response = requests.get(endpoint, params = params)
    res = response.json()
    # Extract budget, revenue, and genres
    budgets.append(res['budget'])
    revenues.append(res['revenue'])
    genres.append([genre['name'] for genre in res['genres']])
    # Sleep before next API call
    time.sleep(0.25)

In [22]:
# Convert movie data to a pandas DataFrame
tmdb_movie_data_df = pd.DataFrame({'Movie_Title': movie_titles, 'Release_Year': years, 'Genre': genres, 'Vote_Average': vote_averages, 'Vote_Count': vote_counts, 'Budget': budgets, 'Revenue': revenues, 'TMDB_ID': tmdb_ids})
tmdb_movie_data_df

,Movie_Title,Release_Year,Genre,Vote_Average,Vote_Count,Budget,Revenue,TMDB_ID
0,Avengers: Age of Ultron,2015,"[Action, Adventure, Science Fiction]",7.271,23847,365000000,1405403694,99861
1,Mad Max: Fury Road,2015,"[Action, Adventure, Science Fiction]",7.627,23503,150000000,378858340,76341
2,Inside Out,2015,"[Animation, Family, Adventure, Drama, Comedy]",7.910,22917,175000000,857611174,150540
3,Jurassic World,2015,"[Action, Adventure, Science Fiction, Thriller]",6.700,21094,150000000,1671537444,135397
4,The Martian,2015,"[Drama, Adventure, Science Fiction]",7.690,20579,108000000,631058917,286217
...,...,...,...,...,...,...,...,...
995,Miller's Girl,2024,"[Thriller, Drama, Romance]",6.348,880,4000000,1430985,1026436
996,I'm Still Here,2024,"[Drama, History]",7.951,861,1480000,36109482,1000837
997,Love Lies Bleeding,2024,"[Crime, Romance, Thriller, Drama]",6.572,856,10000000,12778225,948549
998,Arthur the King,2024,"[Adventure, Drama]",7.577,855,19000000,40829138,618588


In [44]:
# Import list of 2015 - 2024 Oscar winners from best_picture.csv
best_picture = pd.read_csv('../data/best_picture.csv')
oscar_winners_2015_2024 = best_picture[(best_picture['Awards_Year'] >= 2015) & (best_picture['Winner'] == 'Yes')]
oscar_winners_2015_2024

,Unnamed: 0,Title,Awards_Year,Winner
520,520,Spotlight,2015,Yes
528,528,Moonlight,2016,Yes
537,537,The Shape of Water,2017,Yes
546,546,Green Book,2018,Yes
554,554,Parasite,2019,Yes
563,563,Nomadland,2020,Yes
571,571,CODA,2021,Yes
581,581,Everything Everywhere All at Once,2022,Yes
591,591,Oppenheimer,2023,Yes
601,601,Anora,2024,Yes


In [53]:
for title in oscar_winners_2015_2024['Title']:
    if title in tmdb_movie_data_df['Movie_Title'].values:
        print(f'{title} is already in the tmdb_movie_data_df DataFrame.')
    else:
        print(f'{title} is NOT in the tmdb_movie_data_df DataFrame.')

Spotlight is already in the tmdb_movie_data_df DataFrame.
Moonlight is already in the tmdb_movie_data_df DataFrame.
The Shape of Water is already in the tmdb_movie_data_df DataFrame.
Green Book is already in the tmdb_movie_data_df DataFrame.
Parasite is already in the tmdb_movie_data_df DataFrame.
Nomadland is already in the tmdb_movie_data_df DataFrame.
CODA is already in the tmdb_movie_data_df DataFrame.
Everything Everywhere All at Once is already in the tmdb_movie_data_df DataFrame.
Oppenheimer is already in the tmdb_movie_data_df DataFrame.
Anora is already in the tmdb_movie_data_df DataFrame.


In [54]:
# Write the tmdb_movie_data_df DataFrame to a csv file in the data folder
filepath = Path('../data/movies_2015_2024.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
tmdb_movie_data_df.to_csv(filepath)  